# Report: Library coverage of SJR titles

#### Description
This notebook allows you to compare list(s) of journal titles from __[Scimago Journal Ranking reports (SJR)](https://www.scimagojr.com/)__ against your library's holdings of journals in Primo. It will generate a spreadsheet of the full-text availability for all titles in the report, automatically!

#### Dependencies & requirements
This notebook queries the Alma link resolver, so this will only work for libraries that use Primo.

To run this notebook, you will need:
* Python 3
* Jupyter Notebooks
* The open source packages that are loaded below

#### Notes & disclaimer
This code may not be perfect, so it is worth double checking the results. Errors will also be introduced based on the metadata quality:
* The ISSNs provided by the SJR report may not match up with the ISSNs in your MARC records
* The coverage availability statements are pulled from the link resolver, which are only as good as your electronic records in Alma

I am always welcome to collaboration -- if this work can be improved, please reach out!

#### Author
Created by Roger Reka and last updated 9 February 2023.

***

# Setup

### Base URL
The first thing you need to do to setup this notebook is to identify the base URL for your Alma link resolver. It should look something like this `https://ca01.alma.exlibrisgroup.com/view/uresolver/01UTON_UW/openurl?`.

Once you have it, go to the `config.py` file and enter it in the `base_URL` field, in between the quotation marks.

You only have to do this once.

### Install the required packages
If you don't have these installed already, you will need to do this now. This notebook uses several open source Python packages that you will have to install into your environment.

* `pandas`
* `requests`
* `xml.etree.ElementTree`
* `re`
* `glob`

You only have to do this once (aside from updates)

***

## Analysis
This is the start of the actual report analysis. 

### Grab your SJR files
Go to __[Scimago Journal Ranking reports (SJR)](https://www.scimagojr.com/journalrank.php)__ webpage and download the ranking lists that you are interested in. 

Place these files in the `/data` folder.

### Run the code
From this section onwards, you can run all the cells below until the report is generated.

In [ ]:
# Load the required Python packages. Note, you will have to install these if you have not yet done this before.

import pandas as pd
import requests
import xml.etree.ElementTree as ElementTree
import re
import glob
import datetime

In [ ]:
# Load the functions from the associated Python file

import autoCollectionsFunctions as cf

In [ ]:
# Find all the CSV files in the directory

files = glob.glob('data/*.csv')
files

### Prepare the data
This section will prepare the data for querying by combining all the data together into one dataframe, and identifying one ISSN for use in querying the link resolver.

In [ ]:
# Create an empty dataframe
df_journals = pd.DataFrame()

In [ ]:
# For every csv file, grab the data from the named columns (only first X rows) and append them to the df_all dataframe. 
# Edit the 'nrows' value to select how many rows from each file should be included. Default is the first 50.

for file in files:
    df_temp = pd.read_csv(file, sep=';', usecols=['Rank', 'Title', 'Type', 'Issn', 'SJR'], nrows=50)
    # Also, add the name of the file to each row
    df_temp['Category'] = file
    df_journals = pd.concat([df_journals, df_temp], sort=False)

In [ ]:
# For those rows with multiple ISSNS (indicated with a comma), delete the second ISSN

df_journals['q_issn'] = df_journals['Issn'].str.split(',').str[0]

In [ ]:
# Select only the columns we need (remove the original ISSN column)

df_journals = df_journals[['Rank', 'Title', 'Type', 'q_issn', 'SJR', 'Category']]

In [ ]:
df_journals

### Query the link resolver
In this section, the notebook will now check every row of the dataframe above and query the ISSN against the Alma link resolver. The link resolver will return a response via structured XML, and the code will parse out the relevant coverage data.

In [ ]:
# Search the Open URL link resolver to find the coverage for these journals

df_journals[['availability', 'coverage']] = df_journals.apply(cf.searchOpenURL ,axis=1)

In [ ]:
# Update the availability statements based on the coverage dates (emabargo, not to the present)

df_journals[['availability']] = df_journals.apply(cf.coverageStatement_availParser,axis=1)

In [ ]:
df_journals

In [ ]:
# Summary

df_journals.availability.value_counts()

In [ ]:
# Export the results into a CSV file

df_journals.to_csv('results/SJR_rankings_report_{}.csv'.format(datetime.datetime.now().strftime("%Y_%m_%d_%H%M%S")))